In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

from database.mongo import Mongo
from pytorch_common.util import LoggerBuilder

import cudf
import torch

In [3]:
LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

1. Import data to **movies** mongodb database:

In [4]:
DATASETS_PATH   = '../datasets'

movies          = cudf.read_json(f'{DATASETS_PATH}/movies_v4.json')
interactions    = cudf.read_json(f'{DATASETS_PATH}/ratings_tags_v2.json')

/opt/miniconda3/envs/thesis/lib/python3.8/site-packages/cudf/io/json.py:62: UserWarning: Using CPU via Pandas to read JSON dataset, this may be GPU accelerated in the future
  warnings.warn(


In [14]:
movies.dtypes

_id                    int64
tmdb_id                int64
imdb_id                int64
title                 object
genres                  list
for_adults            object
budget                 int64
original_language     object
overview              object
poster                object
release               object
popularity           float64
vote_mean            float64
vote_count           float64
tags                    list
dtype: object

In [15]:
movies.head()

,_id,tmdb_id,imdb_id,title,genres,for_adults,budget,original_language,overview,poster,release,popularity,vote_mean,vote_count,tags
0,1,862,114709,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",False,30000000,en,"Led by Woody, Andy's toys live happily in his ...",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,21.946943,7.7,5415.0,"[2009 reissue in stereoscopic 3-d, 3d, 55 movi..."
1,2,8844,113497,Jumanji,"[Adventure, Children, Fantasy]",False,65000000,en,When siblings Judy and Peter discover an encha...,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,17.015539,6.9,2413.0,"[adaptation of book, adapted from:book, advent..."
2,7,11860,114319,Sabrina,"[Comedy, Romance]",False,58000000,en,An ugly duckling having undergone a remarkable...,/jQh15y5YB7bWz1NtffNZmRw0s9D.jpg,1995-12-15,6.677277,6.2,141.0,"[as good maybe better than original, based on ..."
3,4,31357,114885,Waiting to Exhale,"[Comedy, Drama, Romance]",False,16000000,en,"Cheated on, mistreated and stepped on, the wom...",/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,3.859495,6.1,34.0,"[based on novel or book, characters, chick fli..."
4,5,11862,113041,Father of the Bride Part II,[Comedy],False,0,en,Just when George Banks has recovered from his ...,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,8.387519,5.7,173.0,"[aging, baby, clv, comedy, confidence, contrac..."
